# Jupyter Notebook example for MERRIN

In [ ]:
# ==============================================================================
# Imports
# ==============================================================================
from typing import Literal
from pandas import DataFrame

from merrin import MerrinLearner, Observation

## Inputs

### Mandatory

In [ ]:
sbml: str = 'ecoli-small/metabolic_network.sbml'
objective: str = 'Growth'
pkn: str = 'ecoli-small/pkn.txt'
observations_json: str = 'ecoli-small/timeseries_kft.json'

### Optional

In [ ]:
# ~ Solving modes
optimisation_mode: Literal['all', 'subsetmin'] = 'subsetmin'
projection_mode: Literal['network', 'node'] = 'network'

# ~ Solving parameters
lpsolver: Literal['glpk', 'gurobi'] = 'glpk'
nbsol: int = 0 # 0 to enumerate all solution, else the nb of solution to enum
timelimit: float = -1 # -1 if not timelimit, else the timelimit value in second
max_gap: int = 10 # maximum number of timestep than can be added
max_error: float = 0.1 # maximum error rate between observations and predictions
max_clause: int = 20 # maximum number of clauses per rules in DNF

## Preprocessing

### Parse the Prior Knowlege Network file

In [ ]:
parsed_pkn: list[tuple[str, int, str]] = []
with open(pkn, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        # Line format: <Node>\t<Sign>\t<Node>
        split_line: list[str] = line.strip().split()
        node_u: str = split_line[0]
        sign: int = int(split_line[1])
        node_v: str = split_line[2]
        parsed_pkn.append((node_u, sign, node_v))

### Parse Observation json 

In [ ]:
observations: list[Observation] = Observation.load_json(observations_json)

### Select the solving modes

In [ ]:
optimisation: MerrinLearner.Optimisation = MerrinLearner.Optimisation.SUBSETMIN
if optimisation_mode == 'all':
    optimisation = MerrinLearner.Optimisation.ALL

projection: MerrinLearner.Projection = MerrinLearner.Projection.NETWORK
if projection_mode == 'node':
    projection = MerrinLearner.Projection.NODE

## MERRIN

In [ ]:
merrin: MerrinLearner = MerrinLearner(
    sbml, objective, parsed_pkn,   # Mandatory
    max_clause=max_clause          # Optional
)
merrin.set_optimisation(optimisation)
merrin.set_projection(projection)

In [ ]:
rules: DataFrame = merrin.learn(
    observations,                               # Mandatory
    max_gap=max_gap, max_error=max_error,       # Optional
    timelimit=timelimit, lpsolver=lpsolver,     # Optional
    nbsol=nbsol                                 # Optional
)

## Results

In [ ]:
rules